In [73]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
from sklearn.preprocessing import StandardScaler, Imputer, FunctionTransformer, LabelBinarizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
files = [f for f in listdir('./') if '.csv' in f]
files

['df_block0_open26.csv',
 'df_block0_open52.csv',
 'df_block0_open78.csv',
 'test_df_block0_open26.csv',
 'test_df_block0_open52.csv',
 'test_df_block0_open78.csv',
 'test_df_block13_open26.csv',
 'test_df_block13_open52.csv',
 'test_df_block13_open78.csv',
 'test_df_block26_open26.csv',
 'test_df_block26_open52.csv',
 'test_df_block26_open78.csv',
 'test_df_block52_open26.csv',
 'test_df_block52_open52.csv',
 'test_df_block52_open78.csv',
 'train_df_block0_open26.csv',
 'train_df_block0_open52.csv',
 'train_df_block0_open78.csv',
 'train_df_block13_open26.csv',
 'train_df_block13_open52.csv',
 'train_df_block13_open78.csv',
 'train_df_block26_open26.csv',
 'train_df_block26_open52.csv',
 'train_df_block26_open78.csv',
 'train_df_block52_open26.csv',
 'train_df_block52_open52.csv',
 'train_df_block52_open78.csv']

In [3]:
lines = [line.strip() for line in open("./results.txt")]
D = {'block_weeks': [], 'open_weeks': [], 'score': [], 
     'gb__max_depth': [], 'gb__n_estimators': [],
     'logreg__C': [], 'logreg__penalty': [],
     'rf__max_depth': [], 'rf__n_estimators': []} 
#      'sgd_cls__alpha': [], 'sgd_cls__l1_ratio': [], 
#      'sgd_cls__loss': [], 'sgd_cls__penalty': []}

for i, line in enumerate(lines): 
    print(line.strip())
    if i%4 == 0:        
        D['block_weeks'].append(int(line[16:18]))
        D['open_weeks'].append(int(line[23:25]))
        #print(line[16:18])
        #print(line[23:25])
    if i%4 == 1:        
        D['score'].append(float(line[7:]))
        #print(float(line[7:]))
    if i%4 == 3:        
        parameters = line[1:-1].split(', ')
        D['gb__max_depth'].append(int(parameters[0].split(': ')[1]))
        D['gb__n_estimators'].append(int(parameters[1].split(': ')[1]))
        D['logreg__C'].append(float(parameters[2].split(': ')[1]))
        D['logreg__penalty'].append(parameters[3].split(': ')[1])
        D['rf__max_depth'].append(int(parameters[4].split(': ')[1]))
        D['rf__n_estimators'].append(int(parameters[5].split(': ')[1]))
#         D['sgd_cls__alpha'].append(float(parameters[4].split(': ')[1]))
#         D['sgd_cls__l1_ratio'].append(float(parameters[5].split(': ')[1]))
#         D['sgd_cls__loss'].append(parameters[6].split(': ')[1])
#         D['sgd_cls__penalty'].append(parameters[7].split(': ')[1])
        print()
    if i%12 == 11:
        print()

./train_df_block13_open26.csv	./test_df_block13_open26.csv
score: 0.873134328358
best parameters:
{'gb__max_depth': 5, 'gb__n_estimators': 500, 'logreg__C': 0.1, 'logreg__penalty': 'l1', 'rf__max_depth': 5, 'rf__n_estimators': 100}

./train_df_block13_open52.csv	./test_df_block13_open52.csv
score: 0.869402985075
best parameters:
{'gb__max_depth': 5, 'gb__n_estimators': 500, 'logreg__C': 1.0, 'logreg__penalty': 'l1', 'rf__max_depth': 5, 'rf__n_estimators': 100}

./train_df_block13_open78.csv	./test_df_block13_open78.csv
score: 0.865671641791
best parameters:
{'gb__max_depth': 5, 'gb__n_estimators': 500, 'logreg__C': 0.1, 'logreg__penalty': 'l2', 'rf__max_depth': 5, 'rf__n_estimators': 100}


./train_df_block26_open26.csv	./test_df_block26_open26.csv
score: 0.850746268657
best parameters:
{'gb__max_depth': 5, 'gb__n_estimators': 500, 'logreg__C': 1.0, 'logreg__penalty': 'l1', 'rf__max_depth': 7, 'rf__n_estimators': 500}

./train_df_block26_open52.csv	./test_df_block26_open52.csv
score: 0

In [12]:
train = pd.read_csv('train_df_block0_open78.csv')
test = pd.read_csv('test_df_block0_open78.csv')
total = train.append(test, ignore_index=True)

In [28]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

chose_b = '13'
chose_o = '78'
train = pd.read_csv('train_df_block0_open'+chose_o+'.csv')
test = pd.read_csv('test_df_block0_open'+chose_o+'.csv')
df_total = train.append(test, ignore_index=True)
# total = './df_block0_open'+chose_o+'.csv'
# df_total = pd.read_csv(total)
unwanted = ['yelp_id', 'name', 'first_review', 'last_review', 'permanently_closed']
wanted = [i for i in df_total.columns if i not in unwanted]
X = df_total[wanted]

block = ['13', '26', '52']
openw = ['26', '52', '78']
for b in block:
    accuracy = []
    for o in openw:
        train = './train_df_block'+b+'_open'+o+'.csv'
        test = './test_df_block'+b+'_open'+o+'.csv'
        print('block weeks: ', b, '\topen weeks: ', o)
        df_train = pd.read_csv(train)
        df_test = pd.read_csv(test)
        X_train = df_train[wanted]
        X_test = df_test[wanted]
        ytrain = df_train['permanently_closed']
        ytest = df_test['permanently_closed']
        imputer = Imputer(missing_values=np.nan, strategy='mean', axis=1)
        ss = StandardScaler() 
        pipe = make_pipeline(imputer, ss) 
        Xtrain = pipe.fit_transform(X_train)
        Xtest = pipe.transform(X_test)

        list_of_classifiers = [
            ('logreg', LogisticRegression(C=0.1, penalty='l2')),
            ('rf', RandomForestClassifier(max_depth=5, n_estimators=100)),
            ('gb', GradientBoostingClassifier(max_depth=5, n_estimators=500))
        ]
        vclf = VotingClassifier(list_of_classifiers, voting='soft')
        vclf.fit(Xtrain, ytrain)
        ypred = vclf.predict(Xtest)
        acc = accuracy_score(ytest, ypred)
        accuracy.append(acc)
        confusion = np.array(confusion_matrix(ytest, ypred))
        print('accuracy: ', acc)
        print('confusion matrix: ')
        print(confusion)
        print(classification_report(ytest, ypred))
        
        if b == chose_b and o == chose_o:
            probability = vclf.predict_proba(X)
            prediction = pd.DataFrame(probability, columns=['non_prob', 'probability'])
            yelp_id = df_total[['yelp_id']]
            yelp_id.join(prediction[['probability']]).to_csv('predictions.csv', index=False)
            
    print('Mean accuracy for '+b+' weeks: ', sum(accuracy)/float(len(accuracy)))
    print()
    print()

block weeks:  13 	open weeks:  26
accuracy:  0.873134328358
confusion matrix: 
[[194   7]
 [ 27  40]]
             precision    recall  f1-score   support

          0       0.88      0.97      0.92       201
          1       0.85      0.60      0.70        67

avg / total       0.87      0.87      0.87       268

block weeks:  13 	open weeks:  52
accuracy:  0.861940298507
confusion matrix: 
[[194   7]
 [ 30  37]]
             precision    recall  f1-score   support

          0       0.87      0.97      0.91       201
          1       0.84      0.55      0.67        67

avg / total       0.86      0.86      0.85       268

block weeks:  13 	open weeks:  78
accuracy:  0.873134328358
confusion matrix: 
[[195   6]
 [ 28  39]]
             precision    recall  f1-score   support

          0       0.87      0.97      0.92       201
          1       0.87      0.58      0.70        67

avg / total       0.87      0.87      0.86       268

Mean accuracy for 13 weeks:  0.869402985075


blo

/Users/cysung/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


accuracy:  0.858208955224
confusion matrix: 
[[195   6]
 [ 32  35]]
             precision    recall  f1-score   support

          0       0.86      0.97      0.91       201
          1       0.85      0.52      0.65        67

avg / total       0.86      0.86      0.85       268

block weeks:  26 	open weeks:  52
accuracy:  0.873134328358
confusion matrix: 
[[194   7]
 [ 27  40]]
             precision    recall  f1-score   support

          0       0.88      0.97      0.92       201
          1       0.85      0.60      0.70        67

avg / total       0.87      0.87      0.87       268

block weeks:  26 	open weeks:  78
accuracy:  0.884328358209
confusion matrix: 
[[195   6]
 [ 25  42]]
             precision    recall  f1-score   support

          0       0.89      0.97      0.93       201
          1       0.88      0.63      0.73        67

avg / total       0.88      0.88      0.88       268

Mean accuracy for 26 weeks:  0.871890547264


block weeks:  52 	open weeks:  26
acc

In [29]:
vclf.classes_

array([0, 1])

In [74]:
p = pd.read_csv('predictions.csv')
p = p.copy()
p['probability'] = p['probability'].apply(lambda x: int(x*100))
p.to_csv('probability.csv', index=False)

In [75]:
prob = pd.read_csv('probability.csv')
prob.head()

,yelp_id,probability
0,afghan-kabob-chicago,11
1,taco-and-burrito-house-chicago,36
2,meatball-hero-and-pasta-chicago,32
3,sinha-elegant-cuisine-chicago,19
4,bella-luna-bar-and-pizza-chicago,11


In [76]:
prob.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1072 entries, 0 to 1071
Data columns (total 2 columns):
yelp_id        1072 non-null object
probability    1072 non-null int64
dtypes: int64(1), object(1)
memory usage: 16.8+ KB


In [77]:
db = pd.read_csv('../DataCollection/database.csv')
droplist = ['probability'] #'phone', 'url', 'claimed_status', 'website', 'address', 'longitude', 'latitude']
db.drop(droplist, inplace=True, axis=1)
db.head()

,yelp_id,name,category,price_range,address,neighborhood,info,Claimed?,website,phone,...,Delivery,Has TV,Attire,Parking,Alcohol,Noise Level,Wi-Fi,ratings_notation,address1,address2
0,afghan-kabob-chicago,Afghan Kabob,"Middle Eastern, Afghan",$11-30,"4040 W Montrose Ave Chicago, IL 60641",Albany Park,"{'Takes Reservations': 'No', 'Delivery': 'Yes'...",1,http://www.afghan-kabob.com,(773) 427-5041,...,Yes,Yes,Casual,Street,No,Quiet,No,4-half,4040 W Montrose Ave,"Chicago, IL 60641"
1,taco-and-burrito-house-chicago,Taco & Burrito House,Mexican,Under $10,"1548 W Fullerton Ave Chicago, IL 60614","Lincoln Park, DePaul","{'Takes Reservations': 'No', 'Delivery': 'No',...",0,NaN,(773) 665-8389,...,No,No,Casual,Street,No,Average,No,3-half,1548 W Fullerton Ave,"Chicago, IL 60614"
2,meatball-hero-and-pasta-chicago,Meatball Hero & Pasta,"American (Traditional), Italian",Under $10,"3037 N Clark St Chicago, IL 60657",Lakeview,"{'Takes Reservations': 'No', 'Delivery': 'Yes'...",1,http://www.meatballheroes.com/,(773) 348-3037,...,Yes,No,Casual,Street,No,Average,Free,4-half,3037 N Clark St,"Chicago, IL 60657"
3,sinha-elegant-cuisine-chicago,Sinha Elegant Cuisine,Brazilian,$11-30,"2018 W Adams Chicago, IL 60612",Near West Side,"{'Takes Reservations': 'Yes', 'Delivery': 'No'...",1,http://www.sinhaelegantcuisine.com,(312) 491-8200,...,No,No,Casual,"Street, Private Lot",No,Average,Free,4-half,2018 W Adams,"Chicago, IL 60612"
4,bella-luna-bar-and-pizza-chicago,Bella Luna Bar & Pizza,"Italian, Pizza",$11-30,"731 N Dearborn Chicago, IL 60654",Near North Side,"{'Takes Reservations': 'Yes', 'Delivery': 'Yes...",1,http://www.bellalunachicago.com/,(312) 751-2552,...,Yes,Yes,Casual,Street,Full Bar,Quiet,Free,3,731 N Dearborn,"Chicago, IL 60654"


In [78]:
db.join(prob[['probability']]).to_csv('database.csv', index=False)

In [80]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1072 entries, 0 to 1071
Data columns (total 65 columns):
yelp_id                      1072 non-null object
name                         1072 non-null object
category                     1070 non-null object
price_range                  1047 non-null object
address                      1072 non-null object
neighborhood                 1061 non-null object
info                         1072 non-null object
Claimed?                     1072 non-null int64
website                      842 non-null object
phone                        1058 non-null object
working_hours                1072 non-null object
ratings                      1072 non-null float64
reviews                      1072 non-null float64
HasWebsite                   1072 non-null int64
first_review                 1072 non-null object
last_review                  1072 non-null object
permanently_closed           1072 non-null int64
1_star                       1072 non-null in

In [81]:
db_new = pd.read_csv('database.csv')
db_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1072 entries, 0 to 1071
Data columns (total 66 columns):
yelp_id                      1072 non-null object
name                         1072 non-null object
category                     1070 non-null object
price_range                  1047 non-null object
address                      1072 non-null object
neighborhood                 1061 non-null object
info                         1072 non-null object
Claimed?                     1072 non-null int64
website                      842 non-null object
phone                        1058 non-null object
working_hours                1072 non-null object
ratings                      1072 non-null float64
reviews                      1072 non-null float64
HasWebsite                   1072 non-null int64
first_review                 1072 non-null object
last_review                  1072 non-null object
permanently_closed           1072 non-null int64
1_star                       1072 non-null in

In [46]:
df = pd.read_csv('train_df_block13_open26.csv')

In [47]:
df.head()

,yelp_id,name,Claimed?,HasWebsite,first_review,last_review,permanently_closed,1_star,2_star,3_star,...,Parking_Garage,Parking_Street,Parking_Validated,Alcohol_Full Bar,Alcohol_No,Noise Level_Loud,Noise Level_Quite,Noise Level_Average,Wi-Fi_No,Wi-Fi_Free
0,afghan-kabob-chicago,Afghan Kabob,1,1,2007-11-20 00:00:00,2017-08-24 00:00:00,0,6,7,14,...,0,1,0,0,1,0,0,0,1,0
1,taco-and-burrito-house-chicago,Taco & Burrito House,0,0,2006-02-11 00:00:00,2017-08-26 00:00:00,0,13,13,12,...,0,1,0,0,1,0,0,1,1,0
2,meatball-hero-and-pasta-chicago,Meatball Hero & Pasta,1,1,2015-07-08 00:00:00,2016-06-21 00:00:00,1,1,1,0,...,0,1,0,0,1,0,0,1,0,1
3,sinha-elegant-cuisine-chicago,Sinha Elegant Cuisine,1,1,2006-12-26 00:00:00,2017-08-11 00:00:00,0,5,5,12,...,0,0,0,0,1,0,0,1,0,1
4,bella-luna-bar-and-pizza-chicago,Bella Luna Bar & Pizza,1,1,2005-07-22 00:00:00,2017-08-01 00:00:00,0,25,18,19,...,0,1,0,1,0,0,0,0,0,1


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804 entries, 0 to 803
Columns: 229 entries, yelp_id to Wi-Fi_Free
dtypes: float64(11), int64(214), object(4)
memory usage: 1.4+ MB


In [49]:
df.columns

Index(['yelp_id', 'name', 'Claimed?', 'HasWebsite', 'first_review',
       'last_review', 'permanently_closed', '1_star', '2_star', '3_star',
       ...
       'Parking_Garage', 'Parking_Street', 'Parking_Validated',
       'Alcohol_Full Bar', 'Alcohol_No', 'Noise Level_Loud',
       'Noise Level_Quite', 'Noise Level_Average', 'Wi-Fi_No', 'Wi-Fi_Free'],
      dtype='object', length=229)

# Build 9 models for different combination of block weeks = [13, 26, 52] and open weeks = [26, 52, 78] with voting classifier

In [ ]:
file = open("results.txt", "w+")
file.close()

block = ['13', '26', '52']
openw = ['26', '52', '78']
for b in block:
    for o in openw:
        train = './train_df_block'+b+'_open'+o+'.csv'
        test = './test_df_block'+b+'_open'+o+'.csv'
        print(train, test)
        df_train = pd.read_csv(train)
        df_test = pd.read_csv(test)
        unwanted = ['yelp_id', 'name', 'first_review', 'last_review', 'permanently_closed']
        wanted = [i for i in df_train.columns if i not in unwanted]
        X_train = df_train[wanted]
        X_test = df_test[wanted]
        ytrain = df_train['permanently_closed']
        ytest = df_test['permanently_closed']
        imputer = Imputer(missing_values=np.nan, strategy='mean', axis=1)
        ss = StandardScaler() 
        pipe = make_pipeline(imputer, ss) 
        Xtrain = pipe.fit_transform(X_train)
        Xtest = pipe.transform(X_test)

        list_of_classifiers = [
            ('logreg', LogisticRegression()),
        #    ('sgd_cls', SGDClassifier()),
        #     ('dt', DecisionTreeClassifier()),
            ('rf', RandomForestClassifier()),
            ('gb', GradientBoostingClassifier())
        ]
        vclf = VotingClassifier(list_of_classifiers, voting='soft')
        params = {
            'logreg__C': [0.1, 1.0, 10.0],
            'logreg__penalty': ['l1', 'l2'],
#             'sgd_cls__loss':['log'], #, 'hinge'], #, 'squared_loss'],
#             'sgd_cls__penalty':['elasticnet'],
#             'sgd_cls__alpha':np.logspace(-1,1,5),
#             'sgd_cls__l1_ratio':[i/10.0 for i in range(6)],
            #'dt__max_depth': [3, 5, 7, 9],
            'rf__max_depth': [5, 7],
            'rf__n_estimators': [100, 500],
            'gb__max_depth': [5, 7],
            'gb__n_estimators': [100, 500],
        }
        gs = GridSearchCV(vclf, params, verbose=2)    
        gs.fit(Xtrain, ytrain)
        with open("results.txt","a+") as file:
            file.write(train+'\t'+test+'\n')            
            file.write('score: '+str(gs.score(Xtest, ytest))+'\n')            
            file.write('best parameters:'+'\n')
            file.write(str(gs.best_params_))
            file.write('\n')

./train_df_block13_open26.csv ./test_df_block13_open26.csv
Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100, total=   1.1s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100, total=   1.1s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100, total=   1.0s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=500 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=500, total=   1.8s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=500 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=500, total=   2.0s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=5

[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l1, rf__max_depth=7, rf__n_estimators=500, total=   2.4s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l1, rf__max_depth=7, rf__n_estimators=500 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l1, rf__max_depth=7, rf__n_estimators=500, total=   2.6s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=100 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=100, total=   1.0s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=100 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=100, total=   1.3s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=1

[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=100, total=   1.1s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=100 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=100, total=   1.2s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=500 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=500, total=   1.9s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=500 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=500, total=   1.9s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l2, rf__max_depth=7, rf__n_esti

...

[Parallel(n_jobs=1)]: Done 288 out of 288 | elapsed: 479.4min finished


./train_df_block52_open26.csv ./test_df_block52_open26.csv
Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100, total=   0.9s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100, total=   0.9s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100, total=   1.0s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=500 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=500, total=   1.7s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=500 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=500, total=   1.7s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=5

[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l1, rf__max_depth=7, rf__n_estimators=500, total=   2.9s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l1, rf__max_depth=7, rf__n_estimators=500 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l1, rf__max_depth=7, rf__n_estimators=500, total=   2.2s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=100 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=100, total=   1.0s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=100 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=100, total=   1.4s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=1

# Read in results.txt

In [ ]:
lines = [line.strip() for line in open("./results.txt")]
D = {'block_weeks': [], 'open_weeks': [], 'score': [], 
     'gb__max_depth': [], 'gb__n_estimators': [],
     'logreg__C': [], 'logreg__penalty': [],
     'rf__max_depth': [], 'rf__n_estimators': []} 
#      'sgd_cls__alpha': [], 'sgd_cls__l1_ratio': [], 
#      'sgd_cls__loss': [], 'sgd_cls__penalty': []}

for i, line in enumerate(lines): 
    print(line.strip())
    if i%4 == 0:        
        D['block_weeks'].append(int(line[16:18]))
        D['open_weeks'].append(int(line[23:25]))
        #print(line[16:18])
        #print(line[23:25])
    if i%4 == 1:        
        D['score'].append(float(line[7:]))
        #print(float(line[7:]))
    if i%4 == 3:        
        parameters = line[1:-1].split(', ')
        D['gb__max_depth'].append(int(parameters[0].split(': ')[1]))
        D['gb__n_estimators'].append(int(parameters[1].split(': ')[1]))
        D['logreg__C'].append(float(parameters[2].split(': ')[1]))
        D['logreg__penalty'].append(parameters[3].split(': ')[1])
        D['rf__max_depth'].append(int(parameters[4].split(': ')[1]))
        D['rf__n_estimators'].append(int(parameters[5].split(': ')[1]))
#         D['sgd_cls__alpha'].append(float(parameters[4].split(': ')[1]))
#         D['sgd_cls__l1_ratio'].append(float(parameters[5].split(': ')[1]))
#         D['sgd_cls__loss'].append(parameters[6].split(': ')[1])
#         D['sgd_cls__penalty'].append(parameters[7].split(': ')[1])
        print()
    if i%12 == 11:
        print()
    

        
#         parameters = line[1:-1].split(', ')
#         print(int(parameters[1].split(': ')[1]))
        

# Score and best parameters for each model

In [34]:
results = pd.DataFrame(D).transpose()
results.reindex(['block_weeks', 'open_weeks', 'score', 
                 'sgd_cls__alpha', 'sgd_cls__l1_ratio', 'sgd_cls__loss', 'sgd_cls__penalty',
                 'rf__max_depth', 'rf__n_estimators','gb__max_depth', 'gb__n_estimators'])

,0,1,2,3,4,5,6,7,8
block_weeks,13,13,13,26,26,26,52,52,52
open_weeks,26,52,78,26,52,78,26,52,78
score,0.86194,0.850746,0.86194,0.869403,0.839552,0.876866,0.843284,0.843284,0.828358
sgd_cls__alpha,10,10,10,1,3.16228,10,10,3.16228,10
sgd_cls__l1_ratio,0.4,0.1,0.2,0.2,0.1,0.5,0.3,0.1,0.4
sgd_cls__loss,'log','log','log','log','log','log','log','log','log'
sgd_cls__penalty,'elasticnet','elasticnet','elasticnet','elasticnet','elasticnet','elasticnet','elasticnet','elasticnet','elasticnet'
rf__max_depth,5,5,5,7,5,5,7,5,5
rf__n_estimators,500,100,100,100,100,100,500,500,500
gb__max_depth,5,5,5,5,5,5,5,7,7


Block = 13

In [20]:
(0.857638888889 + 0.864583333333 + 0.861111111111)/3

0.861111111111

Block = 26

In [21]:
(0.850694444444 + 0.871527777778 + 0.864583333333)/3

0.8622685185183333

Block = 52

In [22]:
(0.868055555556 + 0.868055555556 + 0.854166666667)/3

0.8634259259263333

# Rerun each model to print accuracy and classification report 

In [78]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

chose_b = '26'
chose_o = '78'
total = './df_block0_open'+chose_o+'.csv'
df_total = pd.read_csv(total)
unwanted = ['yelp_id', 'name', 'first_review', 'last_review', 'permanently_closed']
wanted = [i for i in df_train.columns if i not in unwanted]
X = df_total[wanted]

df_total = pd.read_csv(total)
block = ['13', '26', '52']
openw = ['26', '52', '78']
for b in block:
    accuracy = []
    for o in openw:
        train = './train_df_block'+b+'_open'+o+'.csv'
        test = './test_df_block'+b+'_open'+o+'.csv'
        print('block weeks: ', b, '\topen weeks: ', o)
        df_train = pd.read_csv(train)
        df_test = pd.read_csv(test)
        X_train = df_train[wanted]
        X_test = df_test[wanted]
        ytrain = df_train['permanently_closed']
        ytest = df_test['permanently_closed']
        imputer = Imputer(missing_values=np.nan, strategy='mean', axis=1)
        ss = StandardScaler() 
        pipe = make_pipeline(imputer, ss) 
        Xtrain = pipe.fit_transform(X_train)
        Xtest = pipe.transform(X_test)

        list_of_classifiers = [
            ('logreg', LogisticRegression(C=0.1, penalty='l1')),
            ('rf', RandomForestClassifier(max_depth=7, n_estimators=100)),
            ('gb', GradientBoostingClassifier(max_depth=5, n_estimators=500))
        ]
        vclf = VotingClassifier(list_of_classifiers, voting='soft')
        vclf.fit(Xtrain, ytrain)
        ypred = vclf.predict(Xtest)
        acc = accuracy_score(ytest, ypred)
        accuracy.append(acc)
        confusion = np.array(confusion_matrix(ytest, ypred))
        print('accuracy: ', acc)
        print('confusion matrix: ')
        print(confusion)
        print(classification_report(ytest, ypred))
        
        if b == chose_b and o == chose_o:
            probability = vclf.predict_proba(X)
            prediction = pd.DataFrame(probability, columns=['probability'])
            yelp_id = df_total[['yelp_id']]
            yelp_id.join(prediction).to_csv('predictions.csv', index=False)
            
    print('Mean accuracy for '+b+' weeks: ', sum(accuracy)/float(len(accuracy)))
    print()
    print()

block weeks:  13 	open weeks:  26
accuracy:  0.857638888889
confusion matrix: 
[[201   9]
 [ 32  46]]
             precision    recall  f1-score   support

          0       0.86      0.96      0.91       210
          1       0.84      0.59      0.69        78

avg / total       0.86      0.86      0.85       288

block weeks:  13 	open weeks:  52
accuracy:  0.868055555556
confusion matrix: 
[[199  11]
 [ 27  51]]
             precision    recall  f1-score   support

          0       0.88      0.95      0.91       210
          1       0.82      0.65      0.73        78

avg / total       0.86      0.87      0.86       288

block weeks:  13 	open weeks:  78
accuracy:  0.864583333333
confusion matrix: 
[[202   8]
 [ 31  47]]
             precision    recall  f1-score   support

          0       0.87      0.96      0.91       210
          1       0.85      0.60      0.71        78

avg / total       0.86      0.86      0.86       288

Mean accuracy for 13 weeks:  0.863425925926


blo

# Around 30 closed restaurants are predicted to be open acroos all models:

In [77]:
count = 0
for p, t in zip(ypred, list(ytest)):
    if t == 1 and p == 0:
        count += 1
count

33

In [42]:
a = 1

In [43]:
a == 1

True